In [1]:
import numpy as np
import matplotlib.pyplot as plt
#import scipy as sp
#from scipy.special import jn
#import jcamp as jc
import itertools
import math
import hdbscan
from sklearn.cluster import SpectralClustering
import warnings

In [2]:
'''Noisy IR Spectra Generation Methods'''

#Generate the Data. CALL THIS!
def getNoisyIRSpectra(cleanIR, number_of_trials_per_molecule=100, x_axis_resolution=20000, machine_noise=False, original_gaussian_noise=0.2, max_speed_exponent=7, val_machine_noise=0.05):
    noise_ranges=[original_gaussian_noise, x_axis_resolution/1000, 3*(10 ** max_speed_exponent), val_machine_noise]
    if(machine_noise):
        IR_data_1 = cleanIR#reaxis_multiple_IR(load_all_npy_data(number_of_molecules),x_axis_resolution)
        IR_data_2 = []
        IR_data_2.append(IR_data_1[0])
        for index in range(len(IR_data_1)-1):
            for index2 in range(number_of_trials_per_molecule):
                IR_data_2.append( vertical_noise_addition(gaussian_smoothing(vertical_noise_addition(R_B_shift_data(IR_data_1[index+1],IR_data_1[0],np.random.random()*noise_ranges[2],np.random.random()*2*np.pi),np.random.random()*noise_ranges[0]),noise_ranges[1]),np.random.random()*noise_ranges[3]))
    if(not machine_noise):
        IR_data_1 = cleanIR#reaxis_multiple_IR(convert_all_data(load_all_jc_data(number_of_molecules)),x_axis_resolution)
        IR_data_2 = []
        IR_data_2.append(IR_data_1[0])
        for index in range(len(IR_data_1)-1):
            for index2 in range(number_of_trials_per_molecule):
                IR_data_2.append( gaussian_smoothing(vertical_noise_addition(R_B_shift_data(IR_data_1[index+1],IR_data_1[0],np.random.random()*noise_ranges[2],np.random.random()*2*np.pi),np.random.random()*noise_ranges[0]),noise_ranges[1]))
    return IR_data_2

#Methods for linear interpolation, gaussian smoothing, (vertical) gaussian noise addition, and R/B shift
def interpolate_data_to_new_axis(data,originalAxis,newAxis, style = 2):
     # this method converts points in data from originalAxis to what their values would be at newAxis by interpoliation,
     # style determines how to fill in the points that are beyond the range of originalAxis, 0 fills them in with 0s, 1 fills them in with 1's, 2 filles them in with the last data point
     # any other value fills them in by interpolating the last point via the slope from the second to last point
    newData = np.zeros(newAxis.size, dtype=type(data[0]))
    counter = 0
    for i in range(newAxis.size):
        if newAxis[i]<originalAxis[0]:
            if( style==0):newData[i]=0
            elif(style==1):newData[i]=1
            elif(style==2):newData[i]=data[0]
            else: newData[i]=data[0]+(newAxis[i]-originalAxis[0])*(data[1]-data[0])/(originalAxis[1]-originalAxis[0])
        elif newAxis[i]>originalAxis[-1]:
            if( style==0):newData[i]=0
            elif(style==1):newData[i]=1
            elif(style==2):newData[i]=data[-1]
            else: newData[i]=data[-1]+(newAxis[i]-originalAxis[-1])*(data[-2]-data[-1])/(originalAxis[-2]-originalAxis[-1])
        elif newAxis[i]==originalAxis[counter]:
            newData[i] = data[counter]
        else:
            while originalAxis[counter]<newAxis[i]:
                counter+=1
            newData[i]=data[counter-1]+(newAxis[i]-originalAxis[counter-1])*(data[counter]-data[counter-1])/(originalAxis[counter]-originalAxis[counter-1])
    return(newData)

def R_B_shift( axis1, vel,ang): #This method gives a red/blue shifted axis (using wave numbers, and returns it in wave numbers), takes in velocity in m/s and angle in rad
    c=3e8
    zp1 = (1+vel*np.cos(ang)/c)/np.sqrt(1-vel*vel/(c*c))
    axis2=axis1/zp1
    return(axis2)

def R_B_shift_data(data, axis1, vel,ang):
    return(interpolate_data_to_new_axis(data,axis1,R_B_shift(axis1,vel,ang)))

def gaussian_smoothing(spectrum, noise_param): #this does gaussian smoothing
    #It takes in the spectrum and smooths according to the noise param, which is units of array

    shiftFactor=int(spectrum.size/2) #this is  the center of the array

    runningVect=np.zeros(spectrum.size) #this will be the output vector

    vect = np.array([np.exp(-(j-spectrum.size/2)*(j-spectrum.size/2)/(2*noise_param*noise_param))/np.sqrt(2*np.pi*noise_param*noise_param) for j in range(spectrum.size)])
    #^this sets up a gaussian distribution around the center of the array.

    for i in range(int(spectrum.size/2)):
        runningVect+=np.concatenate((vect[i:],vect[:i]),axis=0)*spectrum[shiftFactor-i]
    for i in range(int(spectrum.size/2)):
        runningVect+=np.concatenate((vect[-(i+1):],vect[:-(i+1)]),axis=0)*spectrum[shiftFactor+(i)]
    return(runningVect)

def vertical_noise_addition(spec,noise): #this adds vertical noise to the spectrum the noise is added as a gaussian centred at 0 with 
    #height given by 'noise', so 'noise' should be in the same units (and relative scale) as spectrum
    return(spec+np.random.normal(0,noise,spec.size))


In [3]:
'''Variance Calculation Methods'''

#Build arrays of Combinations of IR Spectra. CALL THIS FIRST, THEN THE NEXT ONE!
def buildChooseKIRSpectraArrays(file, k):
    full_array = file[0]
    '''Combinations are the combos of numMolecules choose k''' 
    for combination in itertools.combinations(range(1, len(file)), k):
        blockToAdd = file[combination[0]] #Peel off first one so concatenate works
        for element in combination[1:]:
            blockToAdd = np.vstack((blockToAdd, file[element]))
        full_array = np.vstack((full_array, blockToAdd))
    return full_array

#Row 0 will be Frequencies/Wave Numbers. CALL THIS SECOND!
def buildVarArray(combinedData, molCount, k):
    varArray = combinedData[0]
    numberOfCombinations = molChooseK(molCount, k)
    #print('To begin, the SD array has shape:', sdArray.shape, 'And its entries are:', sdArray)
    for i in range(1, numberOfCombinations + 1):
        varArray = np.vstack((varArray, find_variance(selectCombination(combinedData, k, i))))
    #print('Line 1 of sdArray was giving the original IR Spectra and not the SDs of any combinations. Here is what line 1 reads', sdArray[1])
    return varArray

def find_variance(IRSpectra):
    """
    Calculate the variance of IR spectra across samples.
    param: IRSpectra: (p, q) LEAVE OUT THE FREQUENCY ROW
    Returns:
    variance: (q,)
    """
    return np.var(IRSpectra, axis=0)

def selectCombination(combinedData, k, combination):
    rowIndexToStart = 1+(combination-1)*k
    selectedCombination = []
    for row in range(rowIndexToStart, rowIndexToStart + k):
        selectedCombination.append(combinedData[row])
    return selectedCombination

def molChooseK(molCount, k):
    return int(math.factorial(int(molCount))/(math.factorial(int(k))*math.factorial(int(molCount-k))))


In [4]:
'''Comb Placement Methods'''
'''
    Chooses a comb according the following algorithm:
        Consider a (1+(NumMolecules choose SubsetSize))x(NumWaveNumbers) array
        First, place a comb according to the integration method for some arbitrary combination
        (say, the first combination).
        Then, for each comb yet to be placed:
            Sum across the standard deviations of each combination at the already placed combs.
            For whichever combination had the lowest sum,
                place a comb according to the integral method, using that combination's standard deviations.
    You will provide:
    Standard Deviation Array of Combinations of Molecules (will work for all molecules at once)
    Number of Combs to place
    Width of a comb (in wave numbers, typically 10-20)'''
#Returns array of Left Handed Starts. CALL THIS!
def wave_numbers_for_combination_variance(combination_array, numCombs, combWidth):
    '''
    Pass in an array of pairwise standard deviations, WAVE NUMBERS ARE FIRST ROW!!!
    Returns starting indices for combs
    :param combination_array: Array of standard deviations for combinations of molecules WITH WAVE NUMBERS
    :param numCombs: Number of combs to place
    :param combWidth: Width of each comb in WAVE NUMBERS
    :return: Array of left-handed starts
    '''
    #Place the first comb
    indices = integrateRow(combination_array[1], combWidth=combWidth)
    combosCalled = [1]
    #print('Start indices begin as:', indices)
    for i in range(numCombs-1):
        indexOfLowestPair = determineLowestCombination(findVarianceObservedInEachCombination(combination_array, indices, combWidth))
        combosCalled = np.append(combosCalled, indexOfLowestPair)
        indices = np.concatenate((indices, integrateRow(updateDeviations(combination_array[indexOfLowestPair], indices, combWidth), combWidth)))
    #print('The pairs were called in this order:', combosCalled)
    return shiftIndices(indices, combination_array[0][0])

def findVarianceObservedInEachCombination(combination_array, startIndices, combWidth, callingOnlyOnce=False):
    '''
    Sum the variance of each combination of molecules, according to
    whether a comb has been placed at each wave number
    :param combination_array: Full combination array
    :param startIndices: Set of comb starting wave numbers
    :param combWidth: Width of a comb
    :param callingOnlyOnce: Set to true if you want to know how much 
                            variance was observed by each comb
    :return: array of total variance observed by placed combs for each combination
            Index n is combination n+1
    '''
    #combsPlaced = len(startIndices)
    comboNumber = 0
    totalVariancesOfPairs = np.zeros(shape=(len(combination_array)-1))
    for combination in combination_array[1:]:
        totalVariance = 0
        for start in startIndices:
            totalVariance += np.sum(combination[int(start):int(start)+combWidth])
        totalVariancesOfPairs[comboNumber] = totalVariance
        comboNumber += 1
        if callingOnlyOnce == True:
            print('Combination', comboNumber, 'has total variance:', totalVariance)
    return totalVariancesOfPairs

def determineLowestCombination(observedVariances):
    '''
    Finds which molecule combination has seen the lowest observed variance
    :param observedVariances: array of total variance observed by placed combs for each combination
    :return: Returns the lowest combination's index with respect to 
            the original, full combination array (accounts for row of wave numbers)
    '''
    #Adding 1 is necessary due to wave number row
    return np.argmin(observedVariances) + 1

def updateDeviations(combination, startIndices, combWidth):
    '''
    Sets variances at wave numbers which already are taken up by a comb
    to 0 so that they are not considered in the next iteration of the
    integral optimization method
    :param combination: The single row of standard deviations corresponding to
                        the combination being integrated
    :param startIndices: Set of comb starting wave numbers
    :param combWidth: Width of a comb
    :return: Sparse row of standard deviations for a single combination
    '''
    for index in startIndices:
        combination[index: index + combWidth] = 0
    return combination

def integrateRow(combination, combWidth, combsToPlace=1):#, waveNumbersToTruncate):
    '''
    Finds the wave number at which the left hand of a comb should be placed
    to maximize the sum of standard deviations to-be-observed by said comb
    :param combination: The single row of standard deviations corresponding to
                        the (lowest, according to determineLowestCombination method) 
                        combination being integrated
    :param combWidth: (͡°͜ʖ͡°)
    :param combsToPlace: Determines the number times to iterate
                        through integration. Just leave as 1.
    :return: An array of the left handed indices which maximize the integral
            of standard deviations, for the given combination, over a comb width
    '''
    startIndices = []
    for j in range(combsToPlace):
        max_sum = np.sum(combination[0:combWidth-1])
        current_sum = max_sum
        startIndexOfMaximum = 0
        #Iterate through all wave numbers to find index of max interval
        for i in range(0, len(combination) - combWidth + 1):
            current_sum = current_sum - combination[i - 1] + combination[i + combWidth - 1]
            if current_sum > max_sum:
                max_sum = current_sum
                startIndexOfMaximum = i
        startIndices.append(startIndexOfMaximum)
    return startIndices

def shiftIndices(indices, lowestWaveNumber):
    '''
    Ensure outputted left-hand wave number starts is in 
    wave numbers rather than Python Array indices
    :param indices: The starting indices found by integration methodology
    :param lowestWaveNumber: The lowest wave number found in the "frequency" row
                            from the original standard deviation array
    :return: An array of the left handed wave numbers which maximize
     the integral of standard deviations
    '''
    for i in range(len(indices) - 1):
        indices[i] += lowestWaveNumber
    return indices


In [5]:
'''Comb Interaction Methods'''

#Returns an array of shape (# of Noisy IR, Width * 10 * combCount) array.CALL THIS!
def simulateCombInteraction(width, combCount, waveStarts, noisy_IR_spectra, drift_comb=0.000, jitter_comb=0.000, refractive_index_comb = 000.0, absorption_coefficient_comb = 0.0, total_experiment_duration = 1e1, teeth_spacing = 1):
    frequencies = noisy_IR_spectra[0]
    grand_x = np.linspace(0, 4000, 40000, endpoint = False)
    starts = waveStarts[0:combCount]
    export = []
    for stance in range(1, noisy_IR_spectra.shape[0]):
        input_ir_y_axis = noisy_IR_spectra[stance] #These are the IR values to be multiplied against
        transmittance_values = input_ir_y_axis / (np.max(input_ir_y_axis)) # Normalize
        grand_y = np.zeros(len(grand_x)) #This is the row that stores intensities
        #after the interaction (i.e. it is what's saved)
        #For Each Start Index/Each comb
        rowToAdd = []
        for guide in range(len(starts)):
            ### START COMB CREATION
            # Main parameters
            peak_spacing = teeth_spacing
            wavenumber_broadness = 3 * width
            horizontal_comb_shift = find_center(starts[guide], width)
            noise_of_pulse = 0.00
            # Apply parameters
            broadness_of_comb = wavenumber_broadness / 100
            comb_parameters = {'rep_rate': peak_spacing,
                      'pulse_duration': 60e-3 * (1 / broadness_of_comb),
                      'time': total_experiment_duration,
                      'sample_rate': 100e0 * broadness_of_comb,
                      'noise': noise_of_pulse,
                      'jitter': jitter_comb,
                      'drift': drift_comb,
                      'n_0': refractive_index_comb,
                      'alpha_0': absorption_coefficient_comb}
            # Pass input through comb creation functions defined above
            comb_x_axis = comb_x(comb_parameters)
            comb_y_axis = comb_y(comb_parameters, comb_x_axis)
            final_x_axis = comb_x_axis + horizontal_comb_shift
            final_y_axis = comb_y_axis / (np.max(comb_y_axis))
            new_values = trim_data(final_x_axis, final_y_axis, find_center(starts[guide], width), width)
            # Interpolate IR spectra values and simulate interaction with comb
            ir_spectrum_interpolated_values = np.interp(x = new_values[0], xp = frequencies, fp = transmittance_values)
            exiting_comb = ir_spectrum_interpolated_values * new_values[1]
            sorted_indices = np.argsort(new_values[0])
            updated_y = []
            for finality in sorted_indices:
                #updated_x.append(new_values[0][finality])
                updated_y.append(exiting_comb[finality])
            rowToAdd = np.concatenate((rowToAdd, updated_y))
        export.append(rowToAdd)
    return np.asarray(export)

# Define comb creation functions
def comb_x(cps1):
  comb_x_values = np.fft.fftfreq(n = int(cps1['time'] * cps1['sample_rate']), d = 1 / cps1['sample_rate'])
  return comb_x_values

def calculate_h(cps2, comb_x_values_i):
  path_length = 100e-3
  speed_of_light = 3e8
  refractive_index = cps2['n_0']
  absorption_coefficient = cps2['alpha_0']
  refractive_index_transformed = refractive_index + 0.1 * np.sin(comb_x_values_i*2*np.pi)
  absorption_coeffient_transoformed = absorption_coefficient * np.exp(-comb_x_values_i / 1.5e14)
  H_absorption_value = np.exp(-absorption_coeffient_transoformed * path_length)
  H_phase_value = np.exp(-1j * 2 * np.pi * comb_x_values_i * (refractive_index_transformed - 1) * path_length / speed_of_light)
  H_value = H_absorption_value * H_phase_value
  return H_value

def comb_y(cps3, comb_x_values_j):
  # Identify the basic independent variable points representing the entire wave, known as samples
  number_of_samples = int(cps3['time'] * cps3['sample_rate'])
  sample_set = np.zeros(number_of_samples) # Unit is 'number of samples,' representing total amount of points present in the grand train

  # Addresses pulses in the wave
  number_of_pulses_without_reference_to_samples = int(cps3['time'] * cps3['rep_rate'])
  amount_of_samples_coincident_with_pulses = int(cps3['pulse_duration'] * cps3['sample_rate']) # in just one pulse

  # Identify the time points (with units of seconds, not to be confused with sample points) at which pulses start
  pulse_drift_black_box = np.linspace(0,
                                      cps3['drift'] / cps3['rep_rate'],
                                      number_of_pulses_without_reference_to_samples) * np.exp(np.linspace(0,
                                                                                                          100 * cps3['drift'],
                                                                                                          number_of_pulses_without_reference_to_samples))
  pulse_times_noise_black_box = np.random.normal(loc = np.arange(number_of_pulses_without_reference_to_samples) / cps3['rep_rate'],
                                                 scale = cps3['jitter'] / cps3['rep_rate'],
                                                 size = number_of_pulses_without_reference_to_samples)

  # Synthesize to determine pulse time start points
  actual_pulse_time_start_points = np.add(pulse_times_noise_black_box,
                                          pulse_drift_black_box)

  # Wherever sample points are coincident with pulse points, set those sample values to one
  for actual_pulse_time_start_point in actual_pulse_time_start_points:
    starting_sample = int(actual_pulse_time_start_point * cps3['sample_rate'])
    if starting_sample + amount_of_samples_coincident_with_pulses < number_of_samples:
      sample_set[starting_sample:starting_sample + amount_of_samples_coincident_with_pulses] = 1

  # Add noise to all points of the sample train
  sample_set += cps3['noise'] * np.random.normal(size = number_of_samples)

  # Perform Fourier transform on the sample train to identify ampltidues of constituent frequencies
  fourier_amplitudes = np.fft.fft(sample_set)

  # Modify spectrum according to H parameter
  h_parameter = calculate_h(cps3, comb_x_values_j)
  final_amplitudes = fourier_amplitudes * h_parameter
  return np.abs(final_amplitudes)

def find_center(start_freq, first_harmon_width0):
  center = start_freq + (0.5 * first_harmon_width0)
  return center

def trim_data(final_x_axis0, final_y_axis0, horizontal_comb_shift0, width0):
  lower_bound_first_harmonic = horizontal_comb_shift0 - (0.5 * width0)
  upper_bound_first_harmonic = horizontal_comb_shift0 + (0.5 * width0)

  new_x_axis = []
  new_y_axis = []

  for individual in range(len(final_x_axis0)):
    if final_x_axis0[individual] >= lower_bound_first_harmonic and final_x_axis0[individual] < upper_bound_first_harmonic:
      new_x_axis.append(final_x_axis0[individual])
      new_y_axis.append(final_y_axis0[individual])

  grand_update = []
  grand_update.append(new_x_axis)
  grand_update.append(new_y_axis)
  return grand_update


In [6]:
'''Misclustering Methods'''

#CALL THIS
def buildSpectralMisclusterRow(moleculeCount, chooseK, width, combCount, toothSpacing, IR_noise_exponent, combData):
    true_labels = np.repeat(np.arange(moleculeCount), int(len(combData)/moleculeCount))
    return[moleculeCount, chooseK, width, combCount, toothSpacing, IR_noise_exponent, run_spectral_clustering_and_evaluate(combData, true_labels, moleculeCount, n_runs=5)]

#OR THIS
def buildHDBSCANMisclusterRow(moleculeCount, chooseK, width, combCount, toothSpacing, IR_noise_exponent, combData):
    true_labels = np.repeat(np.arange(moleculeCount), int(len(combData)/moleculeCount))
    return[moleculeCount, chooseK, width, combCount, toothSpacing, IR_noise_exponent, run_hdbscan_and_evaluate(combData, true_labels, moleculeCount, n_runs=5)]

def calculate_misclustered_percentage(labels, clusters):
    """
    Calculate the percentage of misclustered points.

    Parameters:
    labels: (m,)
    clusters: (m,)

    Returns:
    misclustered_percentage: scalar
    """
    misclustered_counts = []
    unique_labels = np.unique(labels)
    for label in unique_labels:
        label_indices = labels == label
        cluster_labels, counts = np.unique(clusters[label_indices], return_counts=True)
        majority_count = np.max(counts)
        misclustered_counts.append(np.sum(counts) - majority_count)
    total_misclustered = np.sum(misclustered_counts)
    total_points = len(labels)
    misclustered_percentage = (total_misclustered / total_points) * 100
    return misclustered_percentage

# Runs spectral clustering multiple times and compute the average misclustered percentage
def run_spectral_clustering_and_evaluate(X, labels, n_clusters, n_runs=5):
    """
    Run spectral clustering multiple times and compute the average misclustered percentage.

    Parameters:
    X: (m, n)
    labels: (m,)
    n_clusters: scalar
    n_runs: scalar

    Returns:
    mean_misclustered_percentage: scalar
    """
    misclustered_percentages = []
    for _ in range(n_runs):
        sc = SpectralClustering(n_clusters=n_clusters, affinity='nearest_neighbors', assign_labels='kmeans')
        predicted_labels = sc.fit_predict(X)
        misclustered_percentage = calculate_misclustered_percentage(labels, predicted_labels)
        misclustered_percentages.append(misclustered_percentage)
    misclustered_percentages = sorted(misclustered_percentages)[1:-1]
    return np.mean(misclustered_percentages)

def run_hdbscan_and_evaluate(X, labels, n_clusters, n_runs=5):
    misclustered_percentages = []
    for _ in range(n_runs):
        hdb = hdbscan.HDBSCAN(min_cluster_size=20)
        predicted_labels = hdb.fit_predict(X)
        misclustered_percentage = calculate_misclustered_percentage(labels, predicted_labels)
        misclustered_percentages.append(misclustered_percentage)
    misclustered_percentages = sorted(misclustered_percentages)[1:-1]
    return np.mean(misclustered_percentages)


In [7]:
#Testing on 5 molecules, widths: 10, 20, 30, 40, IR exponent: 1, 2, 3, 4, 5, 6, 7, Tooth Spacing: 1, 2, 5, 10, Comb counts: 1, 2, 3, 4, 5, 10, 15, 20
MOLECULE_COUNT = 10
IR_SPECTRA_FILE = np.load()[0:MOLECULE_COUNT+1] #No noise, from NIST
WIDTHS = [10, 20, 30, 40]
#COMB_COUNTS = [1, 2, 3, 4, 5, 10, 15, 20]
COMB_COUNTS = [1, 2, 3]
#IR_EXPONENTS = [1, 2, 3, 4, 5, 6, 7]
IR_EXPONETS = [1, 2, 3]
#TOOTH_SPACINGS = [1, 2, 5, 10]
TOOTH_SPACINGS = [1, 5, 6, 8, 9, 10, 15, 20]
misclusterArray = []
warnings.filterwarnings('ignore')
for exponent in IR_EXPONENTS:
    noisy_IR_spectra = getNoisyIRSpectra(cleanIR=IR_SPECTRA_FILE, number_of_trials_per_molecule=10,max_speed_exponent=exponent)
    print('Using exponent', str(exponent))
    #for chooseK in range(2, MOLECULE_COUNT + 1):
    for chooseK in [2, 3]:
        variances = buildVarArray(combinedData=buildChooseKIRSpectraArrays(IR_SPECTRA_FILE, chooseK), molCount=MOLECULE_COUNT, k=chooseK)
        print('Using choose', str(chooseK))
        for width in WIDTHS:
            print('Using width', str(width))
            starts = wave_numbers_for_combination_variance(combination_array=variances, numCombs=np.max(COMB_COUNTS), combWidth=width)
            for teeth_spacing in TOOTH_SPACINGS:
                print('Using spacing', teeth_spacing)
                for combCount in COMB_COUNTS:
                    combInteraction = simulateCombInteraction(width=width, combCount=combCount, waveStarts=starts, teeth_spacing=teeth_spacing, noisy_IR_spectra=np.asarray(noisy_IR_spectra))
                    misclusterArray.append(buildSpectralMisclusterRow(moleculeCount=MOLECULE_COUNT, chooseK=chooseK, width=width, combCount=combCount, toothSpacing=teeth_spacing, IR_noise_exponent=exponent, combData=combInteraction))
                print('Done w/ spacing', str(teeth_spacing))
            print('Done w/ width', str(width))
        print('Done w/ choose', str(chooseK))
    print('Done w/ exponent', str(exponent))
                    
np.save(, misclusterArray)

Using exponent 1
Using choose 2
Using width 1
Using spacing 1
Iteration 1 of 17640
Iteration 2 of 17640
Iteration 3 of 17640
Done w/ spacing 1
Using spacing 2
Iteration 4 of 17640
Iteration 5 of 17640
Iteration 6 of 17640
Done w/ spacing 2
Using spacing 3
Iteration 7 of 17640
Iteration 8 of 17640
Iteration 9 of 17640
Done w/ spacing 3
Using spacing 4
Iteration 10 of 17640
Iteration 11 of 17640
Iteration 12 of 17640
Done w/ spacing 4
Using spacing 5
Iteration 13 of 17640
Iteration 14 of 17640
Iteration 15 of 17640
Done w/ spacing 5
Using spacing 6
Iteration 16 of 17640
Iteration 17 of 17640
Iteration 18 of 17640
Done w/ spacing 6
Using spacing 7
Iteration 19 of 17640
Iteration 20 of 17640
Iteration 21 of 17640
Done w/ spacing 7
Using spacing 8
Iteration 22 of 17640
Iteration 23 of 17640
Iteration 24 of 17640
Done w/ spacing 8
Using spacing 9
Iteration 25 of 17640
Iteration 26 of 17640
Iteration 27 of 17640
Done w/ spacing 9
Using spacing 10
Iteration 28 of 17640
Iteration 29 of 17640
It

KeyboardInterrupt: 

In [ ]:
print(misclusterArray)